In [ ]:
import os
import smtplib
from email.header import Header
from email.mime.text import MIMEText
import pandas as pd
import numpy as np

In [ ]:
# импортируем ведомость

marks = pd.read_excel('###', header=0)
marks.head()

In [ ]:
def send_email(recipient: str, msg_text: str, index: int):
    
    login = open('mail.txt', 'r').read()
# в файл mail.txt написать адрес почты, с которой будет рассылка, или же просто написать адрес прямо в эту строку
    password = open('password.txt', 'r').read()
# в файл password.txt написать пароль от почты, с которой будет рассылка, 
# или же просто написать пароль прямо в эту строку

    s = smtplib.SMTP('smtp.yandex.ru', 587, timeout=10)
    s.starttls()
    s.login(login, password)

    
    msg = MIMEText(f'{msg_text}', 'plain', 'utf-8')
    msg['Subject'] = Header('Текущие оценки по курсу "###"', 'utf-8') #тема письма
    msg['From'] = login
    msg['To'] = recipient
    
    
    try:
        s.sendmail(msg['From'], recipient, msg.as_string())
    except Exception as ex:
        print(index) # добавила на случай, если код сломается на середине, и нужно будет отправить оставшиеся
        print(ex)
    finally:
        s.quit()

In [ ]:
for index, row in marks.iterrows():
  
    name = row['name']
    
    cur_marks = []
    text = ''
    for pr in marks.columns[4:]:
        if not np.isnan(row[pr]):
            cur_marks.append(row[pr])
            text += f'Проверочная {pr[-1]}: {int(row[pr])} из 3\n'
    final_text = f'{name}, добрый день! \n\nВаши текущие оценки по курсу "###": \n{text}\nТекущая набранная доля баллов за лекционные проверочные составляет {np.sum(cur_marks)} / 18 ({np.round(np.sum(cur_marks)/18, 2)})\n\nС уважением, \n###'
    send_email(row['почта'], final_text, index)
    